In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import os
import tarfile
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import cv2

tf.config.run_functions_eagerly(True)
# Extract the dataset
dataset_path = 'CUB_200_2011.tgz'
extract_path = 'CUB_200_2011'

with tarfile.open(dataset_path, 'r:gz') as tar:
    tar.extractall(extract_path)

# Load the data
images_path = os.path.join(extract_path, 'CUB_200_2011', 'images')
images_list = []
labels_list = []

with open(os.path.join(extract_path, 'CUB_200_2011', 'images.txt')) as f:
    for line in f:
        images_list.append(line.strip().split(' ')[1])

with open(os.path.join(extract_path, 'CUB_200_2011', 'image_class_labels.txt')) as f:
    for line in f:
        labels_list.append(int(line.strip().split(' ')[1]) - 1)


def load_and_preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128, 128))
    img = img / 255.0
    return img

X = np.array([load_and_preprocess_image(os.path.join(images_path, img_path)) for img_path in images_list])
y = to_categorical(labels_list, num_classes=200)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))  # Additional Conv2D layer
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))  # Additional Conv2D layer
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))
    model.add(layers.Conv2D(256, (3, 3), activation='relu'))  # Additional Conv2D layer
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(200, activation='softmax'))
    
    return model

model = create_cnn_model()
model.summary()


assert model.count_params() < 10_000_000, "The model exceeds 10 million parameters"


model.compile(optimizer=optimizers.Adam(learning_rate=0.0001),  
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


datagen = ImageDataGenerator(
    rotation_range=30,  
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,  
    zoom_range=0.2, 
    horizontal_flip=True)

batch_size = 64  
epochs = 50  

history = model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(X_train) // batch_size,
                    epochs=epochs,
                    validation_data=(X_test, y_test))

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')


c:\Users\nenwa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 59, 59, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 27, 27, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 25, 25, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 10, 10, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 200)            │       102,600 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,363,272 (12.83 MB)

 Trainable params: 3,363,272 (12.83 MB)

...

Epoch 1/50


c:\Users\nenwa\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


147/147 ━━━━━━━━━━━━━━━━━━━━ 785s 5s/step - accuracy: 0.0044 - loss: 5.2989 - val_accuracy: 0.0051 - val_loss: 5.2893
Epoch 2/50
  1/147 ━━━━━━━━━━━━━━━━━━━━ 11:48 5s/step - accuracy: 0.0000e+00 - loss: 5.2893

c:\Users\nenwa\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


147/147 ━━━━━━━━━━━━━━━━━━━━ 87s 564ms/step - accuracy: 0.0000e+00 - loss: 5.2893 - val_accuracy: 0.0076 - val_loss: 5.2884
Epoch 3/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 341s 2s/step - accuracy: 0.0079 - loss: 5.2628 - val_accuracy: 0.0115 - val_loss: 5.1195
Epoch 4/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.0000e+00 - loss: 5.1403 - val_accuracy: 0.0115 - val_loss: 5.1196
Epoch 5/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 118s 796ms/step - accuracy: 0.0091 - loss: 5.1600 - val_accuracy: 0.0119 - val_loss: 5.0273
Epoch 6/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.0000e+00 - loss: 5.1441 - val_accuracy: 0.0127 - val_loss: 5.0270
Epoch 7/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 469s 3s/step - accuracy: 0.0111 - loss: 5.0772 - val_accuracy: 0.0136 - val_loss: 4.9883
Epoch 8/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.0312 - loss: 5.1136 - val_accuracy: 0.0136 - val_loss: 4.9975
Epoch 9/50
147/147 ━━━━━━━━━━━━━━━━━━━━ 130s 876ms/step - accuracy: 0.0101 - loss: 5.0420 